# Trend analysis at Port Hacking

#### Started by Michael Hemming 07/12/2020 in Sydney, NSW

### Description of what this script does:



### Load in Modules:

In [1]:
# general functions
import xarray as xr
import matplotlib as mat
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline  
import numpy as np
# For mann-kendall test and innovative Sen slope analysis
# https://pypi.org/project/pymannkendall/
import pymannkendall as mk
# For pettitt test - need to check results as function copied from stackoverflow
import pettitt as pett

### Load data:


In [2]:
main_path = "\\Users\\mphem\\Documents\\Work\\UNSW\\Trends\\"
NRSPHB_clim = xr.open_dataset(main_path + 'Data\\Port_Hacking_TEMP_Climatology_1953-2019_BottleCTDMooringSatellite.nc')
NRSPHB_agg = xr.open_dataset(main_path + 'Data\\Port_Hacking_TEMP_1953-2019_aggregated.nc')

In [3]:
NRSPHB_agg

<xarray.Dataset>
Dimensions:           (TIME: 95378)
Coordinates:
  * TIME              (TIME) datetime64[ns] 1953-06-09 ... 2019-05-31T12:00:00
Data variables:
    TEMP              (TIME) float32 ...
    TEMP_DATA_SOURCE  (TIME) float32 ...
    DEPTH             (TIME) float32 ...
    LONG              (TIME) float32 ...
    LAT               (TIME) float32 ...
Attributes:
    date_created:                     2020-12-07T10:32:34Z
    title:                            Port Hacking daily temperature timeseri...
    abstract:                         NSW-IMOS Port Hacking daily temperature...
    conventions:                      CF-1.6,IMOS-1.4
    acknowledgement:                  Any users of IMOS data are required to ...
    file_version:                     Level 2 - derived products
    file_version_quality_control:     Data used to produce the climatology ha...
    geospatial_lat_max:               -34.1192
    geospatial_lat_min:               -34.1192
    geospatial_lon_max:               151.2266
    geospatial_lon_min:               151.2266
    geospatial_vertical_max:          100
    geospatial_vertical_min:          0
    geospatial_vertical_positive:     down
    local_time_zone:                  10.0
    climatology_time_zone:            0.0
    principal_investigator:           Roughan, Moninya
    principal_investigator_email:     mroughan@unsw.edu.au
    authors:                          Roughan, M, Hemming, M, Schaeffer, A, A...
    authors_email:                    mroughan@unsw.edu.au
    project:                          Integrated Marine Observing System (IMOS)
    references:                       http://www.imos.org.au
    data_centre:                      Australian Ocean Data Network (AODN)
    data_centre_email:                info@aodn.org.au
    disclaimer:                       Data, products and services from IMOS a...
    distribution_statement:           Data may be re-used, provided that rela...
    license:                          http://creativecommons.org/licenses/by/...
    standard_name_vocabulary:         NetCDF Climate and Forecast (CF) Metada...
    naming_authority:                 IMOS
    climatology_time_coverage_start:  1953-05-31
    climatology_time_coverage_end:    2019-05-31

In [4]:
NRSPHB_clim

<xarray.Dataset>
Dimensions:             (DEPTH: 9, NV: 2, TIME: 365)
Coordinates:
  * TIME                (TIME) datetime64[ns] 1950-01-01T12:00:00 ... 1950-12...
  * DEPTH               (DEPTH) float32 2.0 10.0 20.0 30.0 ... 60.0 75.0 99.0
  * NV                  (NV) float32 nan nan
Data variables:
    TEMP_AVE            (TIME, DEPTH) float32 ...
    TEMP_MED            (TIME, DEPTH) float32 ...
    TEMP_STD            (TIME, DEPTH) float32 ...
    TEMP_MAX            (TIME, DEPTH) float32 ...
    TEMP_MIN            (TIME, DEPTH) float32 ...
    TEMP_PER90          (TIME, DEPTH) float32 ...
    TEMP_PER80          (TIME, DEPTH) float32 ...
    TEMP_PER70          (TIME, DEPTH) float32 ...
    TEMP_PER60          (TIME, DEPTH) float32 ...
    TEMP_PER50          (TIME, DEPTH) float32 ...
    TEMP_PER40          (TIME, DEPTH) float32 ...
    TEMP_PER30          (TIME, DEPTH) float32 ...
    TEMP_PER20          (TIME, DEPTH) float32 ...
    TEMP_PER10          (TIME, DEPTH) float32 ...
    climatology_bounds  (TIME, NV) float32 ...
    NOBS                (TIME, DEPTH) float32 ...
Attributes:
    date_created:                     2020-09-24T14:26:49Z
    title:                            Port Hacking daily temperature climatology
    abstract:                         NSW-IMOS Port Hacking daily temperature...
    conventions:                      CF-1.6,IMOS-1.4
    acknowledgement:                  Any users of IMOS data are required to ...
    file_version:                     Level 2 - derived products
    file_version_quality_control:     Data used to produce the climatology ha...
    geospatial_lat_max:               -34.1192
    geospatial_lat_min:               -34.1192
    geospatial_lon_max:               151.2266
    geospatial_lon_min:               151.2266
    geospatial_vertical_max:          99
    geospatial_vertical_min:          19
    geospatial_vertical_positive:     down
    local_time_zone:                  10.0
    climatology_time_zone:            0.0
    principal_investigator:           Roughan, Moninya
    principal_investigator_email:     mroughan@unsw.edu.au
    authors:                          Roughan, M, Hemming, M, Schaeffer, A, A...
    authors_email:                    mroughan@unsw.edu.au
    project:                          Integrated Marine Observing System (IMOS)
    references:                       http://www.imos.org.au
    data_centre:                      Australian Ocean Data Network (AODN)
    data_centre_email:                info@aodn.org.au
    disclaimer:                       Data, products and services from IMOS a...
    distribution_statement:           Data may be re-used, provided that rela...
    license:                          http://creativecommons.org/licenses/by/...
    standard_name_vocabulary:         NetCDF Climate and Forecast (CF) Metada...
    naming_authority:                 IMOS
    climatology_time_coverage_start:  1953-05-31
    climatology_time_coverage_end:    2019-05-31

### Select data at specific depths (defined when creating the climatology):

In [5]:
# 17-21 m depth
c19 = [(NRSPHB_agg.DEPTH >= 17) & (NRSPHB_agg.DEPTH <= 21)]
D19 = np.array(NRSPHB_agg.DEPTH)
D19 = D19[c19]
t19 = np.array(NRSPHB_agg.TIME)
t19 = t19[c19]
T19 = np.array(NRSPHB_agg.TEMP)
T19 = T19[c19]

<ipython-input-5-5852a3272528>:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  D19 = D19[c19]
<ipython-input-5-5852a3272528>:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  t19 = t19[c19]
<ipython-input-5-5852a3272528>:8: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  T19 = T19[c19]


In [27]:
n = 1

#test = t19[0]
#test
t_grid = []

for n in range(len(time_grid)-1):
    
    t_grid.append(np.datetime64(time_grid[n]))
    c.append([(t19 > time_grid[n]) & (t19 < time_grid[n+1])])
    

np.unique(c)    
    
#format = '%m/%d/%Y %H:%M:%S'
#dt = datetime.strptime(date_string, format)


#c = [(t19 > time_grid[n_bin]) & (t19 < time_grid[n_bin+1])]
#c

array([False])

In [19]:
# Bin data in time

# Create time grid
base = dt.datetime(1953, 1, 1)
time_grid = np.array([base + dt.timedelta(days=i) for i in range(1,24836)])

# binning
for n_bin in range(1,24836):
    
    c = [(t19 > time_grid[n_bin]) & (t19 < time_grid[n_bin]+1)]




ValueError: Could not convert object to NumPy datetime

## Testing functions available

In [97]:
mk_result = mk.original_test(T19)

In [95]:
print(mk_result)

Mann_Kendall_Test(trend='increasing', h=True, p=0.0, z=15.955888333684678, Tau=0.15720560632700817, s=1649884.0, var_s=10692132176.0, slope=0.00038950562778036817, intercept=18.34728590693235)


In [98]:
pett_result = pett.pettitt_test(T19)

C:\Users\mphem\Documents\Work\UNSW\Trends\Scripts\pettitt.py:18: RuntimeWarning: overflow encountered in long_scalars
  p = 2 * np.exp(-6 * K**2 / (T**3 + T**2))


In [99]:
print(pett_result)

(1726, 1.9708311533280896)


In [103]:
print(t19[1726])
# but this is not equal spaced so need to bin/grid first

1984-10-31T23:15:00.000000000


In [108]:
sen_result = mk.sens_slope(T19)
sen_result_season = mk.seasonal_sens_slope(T19)

In [110]:
print(sen_result)
print(sen_result_season)

Sens_Slope_Test(slope=0.00038950562778036817, intercept=18.34728590693235)
Seasonal_Sens_Slope_Test(slope=0.004666744294312729, intercept=18.348683730186337)
